# Non-personalized recommandation

In [1]:
# Imports

import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from itertools import permutations
from time import time
import contextlib

@contextlib.contextmanager
def timer(msg='timer'):
    tic = time()
    yield
    return print(f"{msg}: {time() - tic:.2f}")

In [2]:
movies_df = pd.read_csv("./Data/MovieLens/ml-latest-small/movies.csv", index_col=0)
ratings_df = pd.read_csv("./Data/MovieLens/ml-latest-small/ratings.csv", index_col=0)
user_ratings_df = pd.read_csv("./Data/MovieLens/Processed/user_rating.csv", index_col=0)
user_ratings_df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,1,3,4.0,Grumpier Old Men (1995)
2,1,6,4.0,Heat (1995)
3,1,47,5.0,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,"Usual Suspects, The (1995)"


# Which are the most viewed movies?

In [3]:
user_ratings_df['title'].value_counts().head(20)

Forrest Gump (1994)                                                               329
Shawshank Redemption, The (1994)                                                  317
Pulp Fiction (1994)                                                               307
Silence of the Lambs, The (1991)                                                  279
Matrix, The (1999)                                                                278
Star Wars: Episode IV - A New Hope (1977)                                         251
Jurassic Park (1993)                                                              238
Braveheart (1995)                                                                 237
Terminator 2: Judgment Day (1991)                                                 224
Schindler's List (1993)                                                           220
Fight Club (1999)                                                                 218
Toy Story (1995)                                      

# Which are the best rated movies?

In [4]:
num_ratings = user_ratings_df['title'].value_counts()
mean_ratings = user_ratings_df[['title','rating']].groupby('title').mean().sort_values(by='rating',ascending=False)
mean_ratings['number of ratings'] = num_ratings
mean_ratings.head()

,rating,number of ratings
title,,
Gena the Crocodile (1969),5.0,1
True Stories (1986),5.0,1
Cosmic Scrat-tastrophe (2015),5.0,1
Love and Pigeons (1985),5.0,1
Red Sorghum (Hong gao liang) (1987),5.0,1


The movies with the highest ratings tend to be rather obscure movies with only a few ratings. Let's apply a lower cutoff on the number of ratings to recommend movies that are both highly rated and popular

In [5]:
thr = 50
mean_ratings_thr = mean_ratings[mean_ratings['number of ratings']>thr]
mean_ratings_thr.head(10)

,rating,number of ratings
title,,
"Shawshank Redemption, The (1994)",4.429022,317
"Godfather, The (1972)",4.289062,192
Fight Club (1999),4.272936,218
Cool Hand Luke (1967),4.271930,57
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),4.268041,97
Rear Window (1954),4.261905,84
"Godfather: Part II, The (1974)",4.259690,129
"Departed, The (2006)",4.252336,107
Goodfellas (1990),4.250000,126


# Most commonly seen together items

## Method 1: create a co-occurence matrix using linear algebra (simplest and fastest)

In [6]:
from scipy.sparse import csr_matrix
with timer():
    occ_mat = user_ratings_df[['userId','title','rating']].pivot_table(index='userId', columns='title', values='rating', fill_value=0, aggfunc='count')
    A = csr_matrix(occ_mat.values)
    co_occ_mat2 = A.T@A
plt.imshow(co_occ_mat[:50,:50])
plt.colorbar()

timer: 2.61


NameError: name 'co_occ_mat' is not defined

## Method 2: create a co-occurence matrix by parsing users and movies (more complex (bugprone) and slower)

In [ ]:
# Note: if the data were really large, perm_mat could be built blockwise and converted to sparse matrix format
# we could also take advantage of the fact that the matrix is triangular to that one half doesn't need to be computer or stored
movie_list = user_ratings_df['movieId'].unique()
num_movies = len(movie_list)

MAP_movies = dict(zip(movie_list,range(num_movies)))
co_occ_mat = np.zeros((num_movies, num_movies))
with timer():
    for user in ratings_df.index.unique():
        movies = ratings_df.loc[user,'movieId'].values
        movies = list(map(lambda x: MAP_movies[x], movies))
        for movie in movies:
            co_occ_mat[movie, movies] += 1
        
# Visualize a small portion of the matrix
plt.imshow(co_occ_mat[:50,:50])
plt.colorbar()

## Method 3: Datacamp course method (most complicated and slowest)

In [ ]:
from itertools import permutations

# Create the function to find all permutations
def find_movie_pairs(x):
  pairs = pd.DataFrame(list(permutations(x.values, 2)),
                       columns=['movie_a', 'movie_b'])
  return pairs

# Apply the function to the title column and reset the index
with timer():
    movie_combinations = user_ratings_df.groupby('userId')['title'].apply(find_movie_pairs).reset_index(drop=True)

movie_combinations.head()

In [ ]:
with timer():# Calculate how often each item in movie_a occurs with the items in movie_b
    combination_counts = movie_combinations.groupby(['movie_a', 'movie_b']).size()

    # Convert the results to a DataFrame and reset the index
    combination_counts_df = combination_counts.to_frame(name='size').reset_index()
combination_counts_df.head()

## Verify that the two methods give the same results

In [ ]:
movie_title = 'Jack (1996)'

In [ ]:
# Method 1
movieId = movies_df.loc[movies_df['title']==movie_title].index[0]
matId = MAP_movies[movieId]
temp = pd.DataFrame({'movieId': movie_list, 'size':co_occ_mat[matId,:]})
temp['title'] = movies_df.loc[temp['movieId'],'title'].values
temp.sort_values(by='size',ascending=False).head(10)

In [ ]:
# Method 2
combination_counts_df.loc[pair_counts_df['movie A']==movie_title].sort_values(by='size',ascending=False).head(5)

Both methods yield the same results. Good! Here's a comparison between the two methods:

- Method 1 has the advantage of being fast, both to create the matrix and access the data. Also, it can be stored as an triangular sparse matrix, which would make it memory efficient. The disadvantage is that it needs to be coded by hand and I made quite a few bugs before I managed to set and retrieved the data correctly. Bugs were not easy to find since the data looked reasonable. Another advantage is that the data is easily visualized with a heatmap, and the structure is pretty transparent.

- Method 2 is WAAAY slower than method 1, both to generate the table and to access data. The creation of the table is also quite convoluted. Also, the titles need to be stored, and in the end it will take more space than method 1 (assuming that we would use a sparse matrix format). But there might be less chance of coding bugs because if it's coded improperly, the results will look nothing like what we want.

In [ ]:
with timer():
    

In [ ]:
occ_mat.values.dtype

In [ ]:
occ_mat.shape

In [ ]:
user_ratings_df.shape